with 语句会设置一个临时的上下文，交给上下文管理器对象控制，并且负责清理上下文。

else 子句的行为：
- for
    - 仅当 for 循环运行完毕时（即 for 循环没有被 break 语句中止）才运行 else 块
- while
    - 仅当 while 循环因为条件为假值而退出时（即 while 循环没有被 break 语句中止）才运行 else 块
- try
    - 仅当 try 块中没有异常抛出时才运行 else 块。else 子句抛出的异常不会由前面的 except 子句处理

在所有情况下，如果异常或者 return、break、continue 语句导致控制权跳到了复合语句的主块之外，else 子句也会被跳过。

为了清晰和准确，try 块中应该只抛出预期异常的语句。

在 Python 中，try/except 不仅用于处理错误，还常用于控制流程。

EAFP
- 取得原谅比获得许可容易（easier to ask for forgiveness than permission）
- 先假定存在有效的键或属性，如果假定不成立，那么捕获异常

LBYL
- 三思而后行（look before you leap）
- 在调用函数或查找属性或键之前显示测试前提条件

上下文管理器对象存在的目的是管理 with 语句；迭代器的存在是为了管理 for 语句。

with 语句的目的是简化 try/finally 模式
- 这种模式用于保证一段代码运行完毕后执行某项操作，即便那段代码由于异常、return 语句或 sys.exit() 调用而中止，也会执行指定的操作。
- finally 子句中的代码通常用于释放重要的资源，或者还原临时变更的状态

上下文管理器协议包含 \_\_enter__ 和 \_\_exit__ 两个方法。

执行 with 后面的表达式得到的结果是上下文管理器对象，把值绑定到目标变量上（as 子句）是在上下文管理器对象上调用 \_\_enter__ 方法的结果。

with 语句的 as 子句是可选的。对 open 函数来说，必须加上 as 子句，以便获取文件的引用；有些上下文管理器会返回 None 。

与函数和模块不同，with 块没有定义新的作用域。

In [1]:
# fp 绑定到打开的文件上，__enter__ 方法返回 self
with open('Romeo and Juliet.txt') as fp:
    src = fp.read(60)

len(src)

In [2]:
# 仍然可用
fp

In [3]:
# 可以读取 fp 对象的属性
fp.closed, fp.encoding

In [4]:
# 文件已经关闭，__exit__ 方法
fp.read(60)

\_\_enter__
- 隐式的 self

\_\_exit__
- 隐式的 self
- exc_type
    - 异常类
- exc_value
    - 异常实例
- traceback
    - traceback 对象

在 try/finally 语句的 finally 块中调用 sys.exc_info() 得到的就是 \_\_exit__ 接收的这三个参数。

In [5]:
class LookingGlass:
    
    # 仅 self 参数
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write  # 保存原来的方法
        sys.stdout.write = self.reverse_write  # 猴子补丁，替换成自己的方法
        return 'JABBERWOCKY'  # 返回对象

    def reverse_write(self, text):  # 替换方法
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):  # 异常参数x3
        import sys
        sys.stdout.write = self.original_write  # 还原原来的方法
        if exc_type is ZeroDivisionError:  # 捕获异常
            print('Please DO NOT divide by zero!')
            return True
        # 如果 __exit__ 方法返回 None，或者 True 之外的值，with 块中的任何异常都会向上冒泡

In [6]:
# 上下文管理器是 LookingGlass 类的实例
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')  # 打印的内容反向
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [7]:
what

'JABBERWOCKY'

In [8]:
# 恢复原有函数
print('Back to normal.')

Back to normal.


In [9]:
# 在 with 块之外使用 LookingGlass 类

# 实例化
manager = LookingGlass()
manager

In [10]:
# 调用 __enter__() 方法
monster = manager.__enter__()
monster == 'JABBERWOCKY'

True

In [11]:
monster

'JABBERWOCKY'

In [12]:
manager

In [13]:
# 调用 __exit__() 方法，还原
manager.__exit__(None, None, None)
monster

'JABBERWOCKY'

Jupyter Notebook 里运行的结果不符合预期，直接在终端执行没问题。

![](LookingGlass.png)

上下文管理器示例
- 在 sqlite3 模块中用于管理事物
- 在 threading 模块中用于维护锁、条件和信号
- 为 Decimal 对象的算术运算设置环境
- 为了测试临时给对象打补丁

contextlib 模块提供一些类和其他函数
- closing
    - 构建上下文管理器
- suppress
    - 构建临时忽略指定异常的上下文管理器
- @contextmanager
    - 把简单的生成器函数变成上下文管理器
- ContextDecorator
    - 用于定义基于类的上下文管理器
- ExitStack
    - 能进入多个上下文管理器

在使用 @contextmanager 装饰器的生成器中，yield 语句的作用是把函数的定义体分成两部分
- yield 语句前面的代码在 with 块开始时执行（\_\_enter__）
- yield 语句后面的代码在 with 块结束时执行（\_\_exit__）

In [14]:
import contextlib

# 应用装饰器
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write  # 存储原来的方法

    def reverse_write(text):  # 替换方法
        original_write(text[::-1])

    sys.stdout.write = reverse_write  # 替换
    yield 'JABBERWOCKY'  # 产出一个值，这个值会绑定到 with 语句中 as 子句的目标变量上
    sys.stdout.write = original_write  # 跳出 with 块，继续执行 yield 语句之后的代码，复原

In [15]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [16]:
what

'JABBERWOCKY'

contextlib.contextmanager 装饰器会把函数包装成实现 \_\_enter__ 和 \_\_exit__ 方法的类（\_GeneratorContextManager）
- \_\_enter__
    1. 调用生成器函数，保存生成器对象（这里把它称为 gen）
    2. 调用 next(gen)，执行到 yield 关键字所在的位置
    3. 返回 next(gen) 产出的值，以便把产出的值绑定到 with/as 语句中的目标变量上
- \_\_exit__
    1. 检查有没有把异常传给 exc_type；如果有，调用 gen.throw(exception)，在生成器函数定义体中包含 yield 关键字的那一行抛出异常
    2. 否则，调用 next(gen)，继续执行生成器函数定义体中 yield 语句之后的代码
    

使用 @contextmanager 装饰器时，要把 yield 语句放在 try/finally 语句中（或者放在 with 语句中）。

@contextmanager 装饰器优雅且实用，把三个不同的 Python 特性结合到了一起：函数装饰器、生成器、with 语句。

In [17]:
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write  # 替换

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    msg = ''  # 保存错误消息
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:  # 捕获异常
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write  # 复原
        if msg:
            print(msg)  # 打印错误消息